In [15]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='file:///D:/code/nda-mlops-zoomcamp-2024/week-02/mlruns/1', creation_time=1717343602995, experiment_id='1', last_update_time=1717343602995, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

## Load the data

In [16]:
import pandas as pd

In [17]:
def read_data(url: str) -> pd.DataFrame:
    df = pd.read_parquet(url)
    
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda t: t.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    return df

In [19]:
train_data = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
val_data = read_data("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

print(len(train_data), len(val_data))

2898906 2938060


In [20]:
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, root_mean_squared_error

## Model experiment

In [21]:
df_train = train_data
df_val = val_data

In [22]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)
y_train = df_train[target].values

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)
y_val = df_val[target].values

In [23]:
with mlflow.start_run():
    mlflow.set_tag("developer", "nda")
    mlflow.log_param("train_data_path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
    mlflow.log_param("val_data_path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

    mlflow.log_param("mode", "linear_reg")
    model = LinearRegression()
    model = model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)
    print("mse: ", mse)
    print("rmse: ", rmse)

mse:  66.03789333423961
rmse:  8.126370243487532


In [ ]:
with open('./models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, model), f_out)

## Hyperparameters opitimization

In [24]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

In [25]:
def objective(params):

    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(val, "validation")],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(val)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    
    return { "loss": rmse, 'status': STATUS_OK }
    

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # [exp(-3), exp(0)] - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

**Best hyperparameters**

In [30]:
best_params = {
    'learning_rate': 0.9997943283037245,
    'max_depth': 33,
    'min_child_weight': 7.54030397204494,
    'objective': 'reg:linear',
    'reg_alpha': 0.01184897746903326,
    'reg_lambda': 0.011426820732636738,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=500,
    evals=[(val, "validation")],
    early_stopping_rounds=50
)

y_pred = booster.predict(val)
rmse = root_mean_squared_error(y_val, y_pred)
print(rmse)


C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:43:52] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:5.17636
[1]	validation-rmse:5.14858
[2]	validation-rmse:5.12392
[3]	validation-rmse:5.10517
[4]	validation-rmse:5.09740
[5]	validation-rmse:5.06565
[6]	validation-rmse:5.05962
[7]	validation-rmse:5.05482
[8]	validation-rmse:5.03205
[9]	validation-rmse:5.02944
[10]	validation-rmse:5.01495
[11]	validation-rmse:5.00298
[12]	validation-rmse:4.99522
[13]	validation-rmse:4.99311
[14]	validation-rmse:4.99121
[15]	validation-rmse:4.98816
[16]	validation-rmse:4.98704
[17]	validation-rmse:4.98638
[18]	validation-rmse:4.98343
[19]	validation-rmse:4.98317
[20]	validation-rmse:4.98025
[21]	validation-rmse:4.97879
[22]	validation-rmse:4.97899
[23]	validation-rmse:4.97900
[24]	validation-rmse:4.97935
[25]	validation-rmse:4.97958
[26]	validation-rmse:4.97989
[27]	validation-rmse:4.97951
[28]	validation-rmse:4.97948
[29]	validation-rmse:4.97953
[30]	validation-rmse:4.97930
[31]	validation-rmse:4.97823
[32]	validation-rmse:4.97866
[33]	validation-rmse:4.97920
[34]	validation-rmse:4.9

2024/06/05 23:44:26 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/05 23:44:26 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:44:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


4.985531829853468


**Save the model Artifact**

In [31]:
best_params = {
    'learning_rate': 0.9997943283037245,
    'max_depth': 33,
    'min_child_weight': 7.54030397204494,
    'objective': 'reg:linear',
    'reg_alpha': 0.01184897746903326,
    'reg_lambda': 0.011426820732636738,
    'seed': 42
}

mlflow.log_params(best_params)

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=500,
    evals=[(val, "validation")],
    early_stopping_rounds=50
)

y_pred = booster.predict(val)
rmse = root_mean_squared_error(y_val, y_pred)
mlflow.log_metric("rmse", rmse)

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

print(rmse)


C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:58:09] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:5.17636
[1]	validation-rmse:5.14858
[2]	validation-rmse:5.12392
[3]	validation-rmse:5.10517
[4]	validation-rmse:5.09740
[5]	validation-rmse:5.06565
[6]	validation-rmse:5.05962
[7]	validation-rmse:5.05482
[8]	validation-rmse:5.03205
[9]	validation-rmse:5.02944
[10]	validation-rmse:5.01495
[11]	validation-rmse:5.00298
[12]	validation-rmse:4.99522
[13]	validation-rmse:4.99311
[14]	validation-rmse:4.99121
[15]	validation-rmse:4.98816
[16]	validation-rmse:4.98704
[17]	validation-rmse:4.98638
[18]	validation-rmse:4.98343
[19]	validation-rmse:4.98317
[20]	validation-rmse:4.98025
[21]	validation-rmse:4.97879
[22]	validation-rmse:4.97899
[23]	validation-rmse:4.97900
[24]	validation-rmse:4.97935
[25]	validation-rmse:4.97958
[26]	validation-rmse:4.97989
[27]	validation-rmse:4.97951
[28]	validation-rmse:4.97948
[29]	validation-rmse:4.97953
[30]	validation-rmse:4.97930
[31]	validation-rmse:4.97823
[32]	validation-rmse:4.97866
[33]	validation-rmse:4.97920
[34]	validation-rmse:4.9

2024/06/05 23:58:42 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/06/05 23:58:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:58:42] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [23:58:49] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` o

4.985531829853468


Save the preprocessor as artifact:

In [ ]:
mlflow.log_params(best_params)

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=500,
    evals=[(val, "validation")],
    early_stopping_rounds=50
)

y_pred = booster.predict(val)
rmse = root_mean_squared_error(y_val, y_pred)
mlflow.log_metric("rmse", rmse)

with open("models/preprocessor.b", "wb") as f_out:
    pickle.dump(dv, f_out)

mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

print(rmse)

Load the saved model

In [33]:
import mlflow
logged_model = 'runs:/92025337761b47258bb509b166789a29/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# # Predict on a Pandas DataFrame.
# import pandas as pd
# loaded_model.predict(pd.DataFrame(data))

C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:36:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [34]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 92025337761b47258bb509b166789a29

In [35]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

C:\Users\NDA\miniconda3\Lib\site-packages\xgboost\core.py:160: UserWarning: [00:38:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [36]:
y_pred = xgboost_model.predict(val)